In [2]:
import numpy as np
import torch.nn as nn
import scipy as sp
import math
import string
import re

In [ ]:
N = 3 #Maximum number of children from a node, 3 is to account for x-bar schemas

In [ ]:
class LSTMNode(object): #Cf. https://arxiv.org/pdf/1503.00075.pdf
    def __init__(m, n): #Defines a node of input size m and output size n, analogous to a layer of a neural network
        #What I'd like to implement would be to have a set of weights/bias for each type of sentence node and x-bar schema, allowing for fine-tuning without overfitting on every given node
        self.m = m #Input dimensions
        self.n = n #Output dimensions
        self.w = np.random.rand(4, m, n) #Weights
        self.u = np.random.rand(3, N, n, n) #Recurrence weights
        self.uf = np.random.rand(N, N, n, n) #Recurrence openness weights
        self.b = np.random.rand(4, n) #Bias
        self.c = np.zeros(n) #Memory cell
        self.h = np.zeros(n) #Hidden state
    
    def forward(x, h, c): #h is the array of the hidden states of upstream nodes, c the array of the memory cells
        K = len(h) #The number of upstream nodes, at most N
        
        i = sig(np.dot(w[0], x) + np.sum([np.dot(u[0][i], h[i]) for i in range(K)]) + b[0]) #Sig to be replaced by the proper function
        f = np.array([sig(np.dot(w[1], x) + np.sum([np.dot(uf[i][j], h[j]) for i in range(K)]) + b[1]) for j in range(K)])
        o = sig(np.dot(w[0], x) + np.sum([np.dot(u[2][i], h[i]) for i in range(K)]) + b[0])
        self.u = np.tanh(np.dot(w[3], x) + np.sum([np.dot(u[3][i], h[i]) for i in range(K)]) + b[3])
        
        self.c = np.multiply(i, self.u) + np.sum(np.multiply(f, c), axis=0)
        self.h = np.multiply(o, np.tanh(self.c))

class LSTM(object):
    def __init__(nodes, links):
        self.nodes = nodes #We assume that nodes are sorted downstream
        self.links = links
        self.upstream = [[] for n in nodes]
        
        for l in links:
            self.upstream[l[0]].append(l[1])
    
    def forward(x):
        for n in nodes:
            forward(x, [nodes[m].h for m in upstream[n]], [nodes[m].c for m in upstream[n]])
        return nodes[-1].h